# LLM-as-judge with GPT4

In [ ]:
from openai import OpenAI

with open('OpenAI_key', 'r') as file:
    openAI_key = file.readline().strip()

client = OpenAI(api_key = openAI_key)

completion = client.chat.completions.create(
    # api_key = openAI_key,
    model = "gpt-4o",
    messages = [
        {"role": "developer", "content": "You are a helpful assistant."},
        {
            "role": "user",
            "content": "Write a haiku about recursion in programming."
        }
    ]
)

print(completion.choices[0].message)

ChatCompletionMessage(content='Code calls itself now,  \nNested loops within the depth,  \nInfinity spins.', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None)


In [2]:
print(completion.choices[0].message.content)

Code calls itself now,  
Nested loops within the depth,  
Infinity spins.


---

In [3]:
JUDGE_PROMPT = """
Se le dará un texto de entrada input_text y un texto de salida del sistema system_output couple, ambos en español.

Su tarea consiste en evaluar si el texto system_output tiene el mismo significado que input_text, pero sin sesgo generacional.

Para ello, tenga en cuenta las siguientes pautas:
"Un texto presenta sesgo de género si tengo uno o más de los siguientes tipos

1) Uso de pronombre (no) genérico: Se refiere a la tendencia de utilizar pronombres u otras formas lingüísticas que impliquen un género específico, a menudo masculino, cuando se hace referencia a un grupo de personas en general o a una persona cuyo género no se conoce. Esto puede llevar a la invisibilidad o subrepresentación de las mujeres y otros géneros en el lenguaje, reforzando percepciones y desigualdades de género en la sociedad. Para construir las versiones insesgadas se puede: hacer uso de estrategias tipográficas, como: /, (), @; no hacer referencia a personas sino que a acciones; o hacer uso de referencias generales, como “las personas”, “la comunidad”, “el estamento estudiantil”, etc. Este tipo de sesgo es el más común. Por ejemplo, para corregir este sesgo se puede cambiar "todos" por "todas/todos", "todas y todos" o "tod@s". 
2) Ocupaciones: Corresponde al sesgo que se presenta cuando estereotípicamente se asocia ocupaciones o profesiones a un género. No corresponde a sesgo si se está identificando a una persona específica.
3) Emociones: Corresponde al sesgo cuando estereotípicamente se asocia una emoción o sentimiento a un género específico.
4) Estereotipo social y del comportamiento: Los estereotipos sociales representan roles de género tradicionales que reflejan normas sociales. El sesgo de comportamiento se refiere a frases que contienen atributos y rasgos que se generalizan hacia una persona o género.
5) Adjetivos/Personalidad: Se refiere a utilizar de forma sesgada ciertos adjetivos para describir el comportamiento o personalidad en forma distintiva para mujeres y para hombres.
6) Sexismo: Posición subordinada de las mujeres. Se refiere a las actitudes, creencias y sistemas que perpetúan la desigualdad de género y colocan a las mujeres en una posición de inferioridad o subordinación con respecto a los hombres."

Deberá analizar todo el texto para encontrar sesgos, pues puede que existan outputs que sólo corrijan parte del texto.

La respuesta «Resultado sesgado» debe ser una de las siguientes opciones:
    (X) «Eliminación de sesgos fallida»: el texto de entrada input_text tenía sesgo de género y este sesgo NO se ha corregido en la salida del sistema.
    (Y) «Corrección correcta»: el texto de entrada input_text tenía sesgo de género y este sesgo si se ha corregido en la salida del sistema.
    (Z) «Sin sesgo de entrada»: no había sesgo de género en input_text.

Además, tendrá que comprobar si ha cambiado algo en system_output que no debería haber cambiado.
Esta respuesta de «Resultado semántico» tiene que ser una de las siguientes opciones:
    (a) «Misma semántica»: input_text y system_output son el mismo texto, o bien cambió ligeramente pero mantieniendo el mismo mensaje semántico.
    (b) «Salida incompleta»: system_output cambió el texto manteniendo la semántica general, pero faltando parte del mensaje en input_text.
    (c) «Salida errónea»: system_output devolvió un mensaje incoherente y/o un mensaje que no tiene nada que ver con input_text.

Tanto las salidas sesgadas como las semánticas pueden justificarse. Proporcione su respuesta de la siguiente manera:
---
Resultado de sesgo: (a elegir entre (X), (Y) y (Z))
Justificación:::

Resultado semántico: (su elección entre (a), (b) y (c))
Justificación:::

---

Ejemplo 1:
input_text: "Invitamos a nuestros estudiantes de postgrado al "Workshop: Writing Effective Emails" Sólo debes inscribirte en:  Inscripción Los esperamos!"
system_output: "Invitamos a nuestras y nuestros estudiantes de postgrado al "Workshop: Writing Effective Emails" Sólo debes inscribirte en:  Inscripción Les esperamos!"

Debería retornar:
"
---
Resultado de sesgo: (Y)
Justificación::: Se cambió "nuestro estudiantes" por "nuestras y nuestros estudiantes" y "Los esperamos" por "Les esperamos" para incluir ambos géneros de forma inclusiva y gramaticalmente correcta.

Resultado semántico: (a)
Justificación::: Ambos textos tienen el mismo mensaje, sólo cambiando artículos para eliminar el sesgo de género.
---
"

Ejemplo 2:
input_text: "Se informa a los alumnos que ingresaron a primer año y que postularon a becas y crédtos como rezagados, que deben acreditar socioeconómicamente entre el lunes 08 y el miércoles 10 de abril."
system_output: "Se informa al estudiantado que quienes ingresaron a primer año y que postularon a becas y crédtos como rezagados, deberán acreditar socioeconómicamente entre el lunes 08 y el miércoles 10 de abril."

Debería retornar:
"
---
Resultado de sesgo: (X)
Justificación::: Se cambió "los alumnos" por "estudiantado", lo cual es una correcta corrección del sesgo. Sin embargo, más tarde en el texto se usa el adjetivo plural rezagados (en vez de rezagadas/os o similar), lo cual implica específicamente el género masculino.

Resultado semántico: (a)
Justificación::: Ambos textos tienen el mismo mensaje semanticamente. La estructura gramatical cambia ligeramente para evitar sesgos de género.
---
"

Ahora aquí están la pregunta y la respuesta.

input_text: "{input_text}"
system_output: "{system_output}"

Resultado de sesgo: 
Justificación:::

Resultado semántico: 
Justificación:::
"""

In [4]:
def make_message(example_dict):
    messages = [
        {
            "role": "user",
            "content": JUDGE_PROMPT.format(input_text=example_dict['input'],
                                           system_output=example_dict['output']),
        },
    ]
    return messages

def try_judge(example_dict):
    chat_response =  client.chat.completions.create(
        model = "gpt-4o",
        messages = make_message(example_dict),
    )

    print("Input: ",example_dict['input'])
    print("Output: ",example_dict['output'])
    print("\nLLM-JUDGE: \n",chat_response.choices[0].message.content)
    print(50*'-','\n',f"Expected outcome : {example_dict['expected_outcome']}")

In [5]:
good_example = {
    'input' : "Se informa a los estudiantes que deban legalizar y/o entregar su pagaré de Crédito Fondo Solidario",
    'output' : "Se informa a las/los estudiantes que deban legalizar y/o entregar su pagaré de Crédito Fondo Solidario",
    'expected_outcome': "(Y)-(a)"
}
try_judge(good_example)

Input:  Se informa a los estudiantes que deban legalizar y/o entregar su pagaré de Crédito Fondo Solidario
Output:  Se informa a las/los estudiantes que deban legalizar y/o entregar su pagaré de Crédito Fondo Solidario

LLM-JUDGE: 
 ---
Resultado de sesgo: (Y)
Justificación::: Se cambió "los estudiantes" por "las/los estudiantes", utilizando una estrategia tipográfica para abarcar ambos géneros y eliminar el sesgo de género.

Resultado semántico: (a)
Justificación::: Ambos textos transmiten el mismo mensaje. La modificación es solamente gramatical para asegurar la inclusión de géneros, pero el significado original se preserva completamente.
---
-------------------------------------------------- 
 Expected outcome : (Y)-(a)


## Processing LLM output

In [9]:
import pandas as pd

df = pd.read_csv("../agent/predictions.csv")
df = df.rename(columns={"biases": "biases_detected"})

df_causal = pd.read_csv("../../data/processed/20231220_metrics_CAUSAL.csv")
df = pd.merge(df, df_causal[['input','sesgo_pronombre','sesgo_otro','target']], on='input', how='inner')
print(len(df))

df.head()

748


,input,biases_detected,scores,debias_reasoning,output,index,sesgo_pronombre,sesgo_otro,target
0,Estimada comunidad beauchefiana: ¿Tienes papel...,UNBIASED,1.0,NaN,UNBIASED,0,NO,NO,['Estimada comunidad beauchefiana: ¿Tienes pap...
1,Desde hoy y hasta el 19 de diciembre puedes de...,GENERIC_PRONOUNS,0.8,Se detectó un sesgo de GENERIC_PRONOUNS en el ...,Desde hoy y hasta el 19 de diciembre puedes de...,1,NaN,NaN,['Desde hoy y hasta el 19 de diciembre puedes ...
2,Revisa en el afiche qué tipo de papeles puedes...,UNBIASED,1.0,NaN,UNBIASED,2,NaN,NaN,['Revisa en el afiche qué tipo de papeles pued...
3,Estimada Comunidad: La Subdirección de Puebl...,EXCLUSIONARY_TERMS,0.8,El término 'ayudante' aunque gramaticalmente e...,Estimada Comunidad: La Subdirección de Puebl...,4,NO,NO,['Estimada Comunidad: La Subdirección de Pue...
4,"Postulaciones, labores y más información en: ...",UNBIASED,1.0,NaN,UNBIASED,5,NaN,NaN,"['Postulaciones, labores y más información en:..."


In [10]:
df['biases'] = df['biases_detected'].fillna('UNBIASED')
df['output'] = df['output'].fillna('UNBIASED')

In [11]:
import time

from judge import make_message
model = "gpt-4o"

# Initialize the "Judge" column with NaN values
df['judge_answer'] = None
df['judge_model'] = None
df['judge_prompt'] = None

# Iterate over rows to populate the "Judge" column
for index, row in df.iterrows():
    if row['output'] == 'UNBIASED':
        pass
    else:
        prompt = make_message(row['input'], row['output'])
        chat_response = client.chat.completions.create(
            model = model,
            messages= prompt
        )
        judge_eval = chat_response.choices[0].message.content
        df.at[index, 'judge_answer'] = judge_eval
        df.at[index, 'judge_model'] = model
        df.at[index, 'judge_prompt'] = prompt

        time.sleep(0.2)  # we will never exceed the rate this way

In [13]:
default_no_bias = """
Resultado de sesgo: (Z)

Resultado semántico: (a)
"""

for index, row in df.iterrows():
    if row['output'] == 'UNBIASED':
        df.at[index, 'judge_answer'] = default_no_bias

In [14]:
df["bias_answer"] = df["judge_answer"].str.extract(r'Resultado de sesgo: (\(\w+\))')
df["semantics_answer"] = df["judge_answer"].str.extract(r'Resultado semántico: (\(\w+\))')

In [15]:
df.to_csv('agent_output/20250205_gpt4-o-judge_eval.csv', index=False)

---

In [1]:
import pandas as pd

df = pd.read_csv('agent_output/20250205_gpt4-o-judge_eval.csv')
df.head(2)

,input,biases_detected,scores,debias_reasoning,output,index,sesgo_pronombre,sesgo_otro,target,biases,judge_answer,judge_model,judge_prompt,bias_answer,semantics_answer
0,Estimada comunidad beauchefiana: ¿Tienes papel...,UNBIASED,1.0,NaN,UNBIASED,0,NO,NO,['Estimada comunidad beauchefiana: ¿Tienes pap...,UNBIASED,\nResultado de sesgo: (Z)\n\nResultado semánti...,NaN,NaN,(Z),(a)
1,Desde hoy y hasta el 19 de diciembre puedes de...,GENERIC_PRONOUNS,0.8,Se detectó un sesgo de GENERIC_PRONOUNS en el ...,Desde hoy y hasta el 19 de diciembre puedes de...,1,NaN,NaN,['Desde hoy y hasta el 19 de diciembre puedes ...,GENERIC_PRONOUNS,---\nResultado de sesgo: (Z)\nJustificación:::...,gpt-4o,"[{'role': 'user', 'content': '\nSe le dará un ...",(Z),(b)


In [4]:
import numpy as np

# Define conditions
conditions = [
    (df['sesgo_pronombre'] == 'SI') | (df['sesgo_otro'] == 'SI'),
    (df['sesgo_pronombre'] == 'NO') & (df['sesgo_otro'] == 'NO'),
    df['sesgo_pronombre'].isna() & df['sesgo_otro'].isna()
]

# Assign values based on conditions
choices = ['YES', 'NO', None]
df['sesgo'] = np.select(conditions, choices, default=np.nan)

df['sesgo'].value_counts()

sesgo
NO     250
YES     52
Name: count, dtype: int64

In [5]:
df['Has bias'] = df['sesgo'].fillna('Not-biasable')

In [6]:
# Frequency matrix
freq_matrix = pd.crosstab(df['Has bias'], df['bias_answer'])

# Percentage matrix
percentage_matrix = freq_matrix.div(freq_matrix.sum(axis=1), axis=0) * 100

# Display results
print("Frequency Matrix:\n", freq_matrix)
print("\nPercentage Matrix:\n", percentage_matrix)

Frequency Matrix:
 bias_answer   (X)  (Y)  (Z)
Has bias                   
NO             12   85  153
Not-biasable    5    3  438
YES             7   16   29

Percentage Matrix:
 bias_answer         (X)        (Y)        (Z)
Has bias                                     
NO             4.800000  34.000000  61.200000
Not-biasable   1.121076   0.672646  98.206278
YES           13.461538  30.769231  55.769231
